In [2]:
import re
import requests
import xml.etree.ElementTree as ET
import feedparser
from IPython.display import HTML

# --- Sample Input References ---
input_references = """
[1].	W. C., W. K., Y. F., N. H., Y. L., and H. Y. D. Fang, "Development and validation of an EEG-based real-time emotion recognition system using edge AI computing platform with convolutional neural network system-on-chip design," IEEE Journal on Emerging and Selected Topics in Circuits and Systems, vol. 9, no. 4, pp. 693-702, Dec. 2019. DOI: 10.1109/JETCAS.2019.2951232
[2].	A. M. A. M. and M. M. Hassouneh, "Development of a real-time emotion  recognition system using facial expressions and EEG based on machine learning and deep neural network methods," Informatics in Medicine Unlocked, vol. 20, p. 100372, 2020. DOI: 10.1016/j.imu.2020.100372.
[3].	S. M. H., S. R. M., P. M., and J. J. Turabzadeh, "Facial expression emotion detection for real-time embedded systems," Technologies, vol. 6, no. 1, p. 17, 2018. DOI: 10.3390/technologies6010017.
[4].	S. and N. G. C. Jaiswal, "Robust real-time emotion detection system using CNN architecture," Neural Computing and Applications, vol. 32, no. 9, pp. 4827–4838, 2019. DOI: 10.1007/s00521-019-04564-4.
[5].	A. R. M., B. M., Đ.-J. M., and J. M. M. Kartali, "Real-time algorithms for facial emotion recognition: A comparison of different approaches," in 14th Symposium on Neural Networks and Applications (NEUREL), Belgrade, Serbia, 2018, pp. 1-6. DOI: 10.1109/NEUREL.2018.8587000
[6].	S. K. P. and T. R. K. Gupta, "Facial emotion recognition based real-time learner engagement detection system in online learning context using deep learning models," Multimedia Tools and Applications, vol. 82, no. 8, pp. 11365-11394, 2023. DOI: 10.1007/s11042-022-13908-w.
[7].	S. A. and A. B. A. S. A. Hussain, "A real time face emotion classification and recognition using deep learning model," in Journal of Physics: Conference Series, vol. 1378, no. 2, p. 022095, 2019. DOI: 10.1088/1742-6596/1378/2/022095.
[8].	S. Gupta, "Facial emotion recognition in real-time and static images," in 2nd International Conference on Inventive Systems and Control (ICISC), Coimbatore, India, 2018, pp. 108-111. DOI: 10.1109/ICISC.2018.8399046.
[9].	M. R. M., S. N., B. K., M. U., I. S. U., and R. J. J. Awais, "LSTM-based emotion detection using physiological signals: IoT framework for healthcare and distance learning in COVID-19," IEEE Internet of Things Journal, vol. 7, no. 9, pp. 8749-8756, Sep. 2020. DOI: 10.1109/JIOT.2020.2995504.
[10].	A. R., A. K., and D. S. Jaiswal, "Facial emotion detection using deep learning," in International Conference for Emerging Technology (INCET), Belgaum, India, 2020, pp. 1-6. DOI: 10.1109/INCET49848.2020.9154011.
[11].	K. L., Y. W., J. C. E., and L. X. Zhang, "Real-time video emotion recognition based on reinforcement learning and domain knowledge," IEEE Transactions on Circuits and Systems for Video Technology, vol. 31, no. 12, pp. 4856-4868, Dec. 2021. DOI: 10.1109/TCSVT.2021.3071947.
[12].	F. M. Talaat, "Real-time facial emotion recognition system among children with autism based on deep learning and IoT," Neural Computing and Applications, vol. 35, no. 2, pp. 1817–1830, 2023. DOI: 10.1007/s00521-022-07503-1.
[13].	Y. Jiang, W. Li, M. S. Hossain, M. Chen, A. Alelaiwi, and M. Al-Hammadi, "A snapshot research and implementation of multimodal information fusion for data-driven emotion recognition," Information Fusion, vol. 53, pp. 55–62, 2020. DOI: 10.1016/j.inffus.2019.06.003.
[14].	K. V. d. V. W. and W. W. Bahreini, "A fuzzy logic approach to reliable real-time recognition of facial emotions," Multimedia Tools and Applications, vol. 78, no. 15, pp. 20737–20752, Aug. 2019. DOI: 10.1007/s11042-019-7318-6.
[15].	A. and B. K. Iqbal, "A real-time emotion recognition from speech using gradient boosting," in International Conference on Electrical, Computer and Communication Engineering (ECCE), Cox's Bazar, Bangladesh, 2019, pp. 1-6. DOI: 10.1109/ECACE.2019.8679413.
[16].	M. A. E., B. A. A., S. R., and A. A. Ozdemir, "Real-time emotion recognition from facial expressions using CNN architecture," in Medical Technologies Congress (TIPTEKNO), Izmir, Turkey, 2019, pp. 1-4. DOI: 10.1109/TIPTEKNO.2019.8895062.
[17].	L. M. P., W. E. P., K. I., and M. S. B. Zahara, "The Facial Emotion Recognition (FER-2013) dataset for prediction system of micro-expressions face using the convolutional neural network (CNN) algorithm based Raspberry Pi," in 5th International Conference on Informatics and Computing (ICIC), Semarang, Indonesia, 2020, pp. 1-6. DOI: 10.1109/ICIC47613.2020.8985924.
[18].	E. K., S. C., C. S., and S. M. H. Pranav, "Facial emotion recognition using deep convolutional neural network," in 6th International Conference on Advanced Computing and Communication Systems (ICACCS), Coimbatore, India, 2020, pp. 1-6. DOI: 10.1109/ICACCS48705.2020.9074477.
[19].	A. I., S. N. F. A., A. D., A. E.-S. F., E., and S. A. Siam, "Deploying machine learning techniques for human emotion detection," Computational Intelligence and Neuroscience, vol. 2022, p. 2951934, 2022. DOI: 10.1155/2022/2951934.
[20].	K. S. Mustaqeem, "Deep feature-based speech emotion recognition using the hierarchical ConvLSTM network," Mathematics, vol. 8, no. 8, p. 1328, Aug. 2020. DOI: 10.3390/math8081328.
[21].	P. M., J. B., D. K., P. N. C. S. D., and C. P. B. Ghate, "Optimized intelligent speech signal verification system for identifying authorized users," Engineering & Applied Science Research, vol. 50, no. 3, pp. 233–245, 2023. DOI: 10.14456/easr.2023.20.
[22].	T. M. and K. S. Anvarjon, "Optimized intelligent speech signal verification system for identifying authorized users," Sensors, vol. 20, no. 22, p. 6376, Nov. 2020. DOI: 10.3390/s20226376.
[23].	M. J. G., T. U., M. N. I., M. M. M., and K. M. S. Al Nahian, "Towards artificial intelligence driven emotion-aware fall monitoring framework suitable for elderly people with neurological disorder," in International Conference on Brain Informatics, Padua, Italy, 2020, pp. 452–463. DOI: 10.1007/978-3-030-59277-6_39.
[24].	K. S. Mustaqeem, "A CNN-assisted enhanced audio signal processing for speech emotion recognition," Sensors, vol. 19, no. 23, p. 5080, Nov. 2019. DOI: 10.3390/s19235080.
[25].	V. K., C. P. S., L. A., and J. F. Vielzeuf, "An Occam's Razor view on learning audiovisual emotion recognition with small training sets," in 20th ACM International Conference on Multimodal Interaction, Boulder, CO, USA, 2018, pp. 571–575. DOI: 10.1145/3242969.3265006.
[26].	K. S. Mustaqeem, "Optimal feature selection based speech emotion recognition using two-stream deep convolutional neural network," International Journal of Intelligent Systems, vol. 36, no. 2, pp. 708–731, 2021. DOI: 10.1002/int.22254.
[27].	S. and B. M. H. Modi, "Facial emotion recognition using convolution neural network," in 5th International Conference on Intelligent Computing and Control Systems (ICICCS), Madurai, India, 2021, pp. 616–621. DOI: 10.1109/ICICCS51141.2021.9432202.
[28].	J. H. P. A. and H. D. S. Kim, "The extensive usage of the facial image threshing machine for facial emotion recognition performance," Sensors, vol. 21, no. 6, p. 2231, Mar. 2021. DOI: 10.3390/s21062231.
[29].	"Human emotion detection using DeepFace and artificial intelligence †," Engineering Proceedings, vol. 5, no. 1, p. 78, 2021. DOI: 10.3390/ecsa-8-11491.
[30].	J. Y., K. W. F., Z. Z., B. Y., Z. S., and Z. D. Chen, "Temporal convolutional network-enhanced real-time implicit emotion recognition with an innovative wearable fNIRS-EEG dual-modal system," Electronics, vol. 13, no. 7, p. 1234, Apr. 2024. 
[31].	Xu, Y., Lin, Y. S., Zhou, X., & Shan, X. (2024). Utilizing emotion recognition technology to enhance user experience in real-time. Computing and Artificial Intelligence, 2(1), 1388-1388.
[32].	Bakariya, B., Singh, A., Singh, H., Raju, P., Rajpoot, R., & Mohbey, K. K. (2024). Facial emotion recognition and music recommendation system using CNN-based deep learning techniques. Evolving Systems, 15(2), 641-658.
[33].	Selvan, M. A. (2024). Deep Learning Techniques for Comprehensive Emotion Recognition and Behavioral Regulation.
[34].	Haq, H. B. U., Akram, W., Irshad, M. N., Kosar, A., & Abid, M. (2024). Enhanced real-time facial expression recognition using deep learning. Acadlore Trans. Mach. Learn, 3(1), 24-35.
[35].	Mutawa, A. M., & Hassouneh, A. (2024). Multimodal Real-Time patient emotion recognition system using facial expressions and brain EEG signals based on Machine learning and Log-Sync methods. Biomedical Signal Processing and Control, 91, 105942.  
[36].	Rao, M. S., Kiran, A., Naik, S. L., Kumar, A. P., Swamy, K. K., & Devika, P. (2024, January). Facial Emotion Recognition with Convolutional Neural Networks using DL. In 2024 International Conference on Advancements in Smart, Secure and Intelligent Computing (ASSIC) (pp. 1-5). IEEE.
"""

# --- Helper Function for DOI Formatting ---
def format_doi(doi):
    """Return the DOI URL properly formatted, avoiding double prefix."""
    if doi.lower().startswith("https://doi.org/"):
        return doi
    return "https://doi.org/" + doi

# --- Functions for Preprocessing and DOI Extraction ---
def remove_leading_numbering(reference):
    """Remove any leading numbering (e.g., '[8].' or '[8]') from the reference text."""
    return re.sub(r'^\s*\[\d+\]\.?\s*', '', reference)

def extract_doi(reference):
    """Extract the DOI from a reference string and remove trailing punctuation."""
    doi_match = re.search(r"doi:\s*(\S+)", reference, re.IGNORECASE)
    if doi_match:
        return doi_match.group(1).rstrip(".,;")
    return None

# --- Functions for Metadata Retrieval ---
def get_crossref_details(doi):
    """Query the CrossRef API for DOI metadata."""
    url = f"https://api.crossref.org/works/{doi}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json().get("message", {})
        else:
            return {}
    except Exception as e:
        print(f"Exception while querying DOI {doi}: {e}")
        return {}

def get_arxiv_details(doi):
    """Query the arXiv API for metadata if the DOI is from arXiv."""
    parts = doi.split("/")
    if len(parts) >= 2 and "arxiv" in parts[1].lower():
        arxiv_id = parts[1].replace("arXiv.", "")
        url = f"http://export.arxiv.org/api/query?id_list={arxiv_id}"
        feed = feedparser.parse(url)
        if feed.entries:
            entry = feed.entries[0]
            published_date = entry.get("published", entry.get("updated", "Unknown"))
            year = published_date[:4] if published_date != "Unknown" else "Unknown"
            details = {
                "title": [entry.title],
                "author": [{"given": "", "family": author.name} for author in entry.authors],
                "published-online": {"date-parts": [[year]]},
                "container-title": ["arXiv"]
            }
            return details
    return {}

def get_doi_details(doi):
    """Determine the type of DOI and query the appropriate API."""
    if "arxiv" in doi.lower():
        details = get_arxiv_details(doi)
        if details:
            return details
    return get_crossref_details(doi)

# --- Functions for Reference Type Detection (Non-DOI references) ---
def detect_ref_type(ref_text):
    """Detect the type of a reference based on keywords in the text."""
    ref_lower = ref_text.lower()
    if "thesis" in ref_lower or "dissertation" in ref_lower:
        return "thesis"
    elif "patent" in ref_lower:
        return "patent"
    elif "standard" in ref_lower:
        return "standard"
    elif "private communication" in ref_lower or "unpublished" in ref_lower:
        return "unpublished"
    elif "ed." in ref_lower and "vol." not in ref_lower:
        return "book"
    return "unknown"

def format_book(ref_text, index):
    return f'<p style="white-space: pre;">[{index}]\t{ref_text}</p>'

def format_thesis(ref_text, index):
    return f'<p style="white-space: pre;">[{index}]\t{ref_text}</p>'

def format_patent(ref_text, index):
    return f'<p style="white-space: pre;">[{index}]\t{ref_text}</p>'

def format_unpublished(ref_text, index):
    return f'<p style="white-space: pre;">[{index}]\t{ref_text}</p>'

def format_standard(ref_text, index):
    return f'<p style="white-space: pre;">[{index}]\t{ref_text}</p>'

# --- Helper Functions for Metadata-based Formatting ---
def format_authors(authors):
    if not authors:
        return "Not available"
    author_names = [f"{a.get('given', '').strip()} {a.get('family', '').strip()}" 
                    for a in authors if a.get('given') or a.get('family')]
    if len(author_names) >= 4:
        return f"{author_names[0]} et al."
    return ", ".join(author_names)

def get_year(details, original_ref=""):
    pub_info = details.get("published-print", details.get("published-online", {}))
    date_parts = pub_info.get("date-parts", [[]])
    if date_parts and date_parts[0] and date_parts[0][0]:
        year = str(date_parts[0][0])
    else:
        year = "Unknown"
    if year == "Unknown" and original_ref:
        match = re.search(r'\b(19|20)\d{2}\b', original_ref)
        if match:
            year = match.group(0)
    return year

def format_title(title):
    normalized = " ".join(title.split())
    return normalized.title()

def format_citation(index, doi, details, original_reference):
    """
    Return a formatted citation string in HTML format.
    If DOI metadata is missing but DOI exists, split the original reference to remove the DOI part and add a separate DOI line.
    """
    if not details:
        ref_without_doi = re.split(r"\bdoi:\s*", original_reference, flags=re.IGNORECASE)[0].strip().rstrip(",")
        doi_line = f"<br>DOI: {format_doi(doi)}" if doi else ""
        citation = f'<p style="white-space: pre;">[{index}]\t{ref_without_doi}.{doi_line}</p>'
        return citation

    authors = format_authors(details.get("author", []))
    raw_title = details.get("title", ["No title found"])[0]
    title = format_title(raw_title)
    year = get_year(details, original_reference)
    container = details.get("container-title", [])
    source = container[0] if container else ""
    volume = details.get("volume", "Not available")
    issue = details.get("issue", "Not available")
    page = details.get("page", "")
    
    doi_lower = doi.lower() if doi else ""
    source_lower = source.lower() if source else ""
    
    # For Data in Brief or similar - check for "article-number" field.
    if "data in brief" in source_lower or "article-number" in details:
        article_no = details.get("article-number", page)
        citation = f'<p style="white-space: pre;">[{index}]\t{authors}, “{title},” <i>{source}</i>'
        if volume != "Not available":
            citation += f", vol. {volume}"
        if article_no:
            citation += f", Article no. {article_no}"
        if year:
            citation += f", {year}"
        citation += f".<br>DOI: {format_doi(doi)}</p>"
    # For conference papers - include event info.
    elif "conference" in source_lower or (volume == "Not available" and page):
        citation = f'<p style="white-space: pre;">[{index}]\t{authors}, “{title},” {source}'
        if page:
            citation += f", pp. {page}"
        if year:
            citation += f", {year}"
        citation += f".<br>DOI: {format_doi(doi)}</p>"
    # For arXiv preprints.
    elif "arxiv" in doi_lower:
        citation = f'<p style="white-space: pre;">[{index}]\t{authors}, “{title}.”<br>DOI: {format_doi(doi)}</p>'
    # Default: Journal article formatting.
    else:
        citation = f'<p style="white-space: pre;">[{index}]\t{authors}, “{title},” <i>{source}</i>'
        if volume != "Not available":
            citation += f", vol. {volume}"
        if issue != "Not available":
            citation += f", no. {issue}"
        if page:
            citation += f", pp. {page}"
        if year:
            citation += f", {year}"
        citation += f".<br>DOI: {format_doi(doi)}</p>"
    return citation

# --- Formatting Functions for Non-DOI Types ---
def format_book(ref_text, index):
    return f'<p style="white-space: pre;">[{index}]\t{ref_text}</p>'

def format_thesis(ref_text, index):
    return f'<p style="white-space: pre;">[{index}]\t{ref_text}</p>'

def format_patent(ref_text, index):
    return f'<p style="white-space: pre;">[{index}]\t{ref_text}</p>'

def format_unpublished(ref_text, index):
    return f'<p style="white-space: pre;">[{index}]\t{ref_text}</p>'

def format_standard(ref_text, index):
    return f'<p style="white-space: pre;">[{index}]\t{ref_text}</p>'

def detect_ref_type(ref_text):
    ref_lower = ref_text.lower()
    if "thesis" in ref_lower or "dissertation" in ref_lower:
        return "thesis"
    elif "patent" in ref_lower:
        return "patent"
    elif "standard" in ref_lower:
        return "standard"
    elif "private communication" in ref_lower or "unpublished" in ref_lower:
        return "unpublished"
    elif "ed." in ref_lower and "vol." not in ref_lower:
        return "book"
    return "unknown"

# --- Main Processing ---
references_list = [ref.strip() for ref in input_references.strip().split("\n") if ref.strip()]
formatted_citations = []
issues = []

for i, ref in enumerate(references_list, start=1):
    clean_ref = remove_leading_numbering(ref)
    doi = extract_doi(clean_ref)
    if doi:
        details = get_doi_details(doi)
        if details:
            formatted = format_citation(i, doi, details, clean_ref)
            formatted_citations.append(formatted)
        else:
            ref_without_doi = re.split(r"\bdoi:\s*", clean_ref, flags=re.IGNORECASE)[0].strip().rstrip(",")
            formatted_citations.append(f'<p style="white-space: pre;">[{i}]\t{ref_without_doi}.<br>DOI: {format_doi(doi)}</p>')
            issues.append(f"[{i}] could not find the DOI metadata for DOI: {doi}")
    else:
        ref_type = detect_ref_type(clean_ref)
        if ref_type == "book":
            formatted = format_book(clean_ref, i)
        elif ref_type == "thesis":
            formatted = format_thesis(clean_ref, i)
        elif ref_type == "patent":
            formatted = format_patent(clean_ref, i)
        elif ref_type == "unpublished":
            formatted = format_unpublished(clean_ref, i)
        elif ref_type == "standard":
            formatted = format_standard(clean_ref, i)
        else:
            formatted = f'<p style="white-space: pre;">[{i}]\t{clean_ref}</p>'
        formatted_citations.append(formatted)

if issues:
    issues_section = "<h2>Issues</h2>" + "\n".join(f"<p>{issue}</p>" for issue in issues)
else:
    issues_section = ""

html_output = "<html><body>" + "\n".join(formatted_citations) + issues_section + "</body></html>"

# Render the HTML in a Jupyter Notebook.
HTML(html_output)

# Alternatively, print the HTML code for copying:
# print(html_output)
